In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments

from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
import torch
import wandb


lora_r: int = 256            # 행렬의 랭크
lora_dropout: float = 0.1  # LoRA parameter에 적용할 dropout 확률
lora_alpha: int = 32       # LoRA parameter인 $A, B$ 행렬을 scaling할 때 사용하는 값

wandb.init(project='Hanghae99', name=f"rank {lora_r}")


dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")


# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")

target_modules = set()

for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        names = name.split('.')
        target_modules.add(names[0] if len(names) == 1 else names[-1])

if "lm_head" in target_modules:  # needed for 16-bit
    target_modules.remove("lm_head")

target_modules = list(target_modules)

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=target_modules
)
model = get_peft_model(model, peft_config)


# 'formatting_prompts_func'는 데이터셋 예시를 입력 받아, 'Instruction'과 'Output'을 적절한 형식으로 변환합니다.
# 각 'Instruction'과 'Output' 쌍을 연결하여 모델이 이를 처리할 수 있도록 합니다.
# 주어진 형식: '### Question: [Instruction]\n### Answer: [Output]'
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"

collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


from transformers import TrainerCallback, TrainerState, TrainerControl

# 콜백 클래스 정의
class WandbLoggingCallback(TrainerCallback):
    def on_log(self, args, state: TrainerState, control: TrainerControl, logs=None, **kwargs):
        if logs is not None:
            # train loss 기록
            if "loss" in logs:
                wandb.log({"train/loss": logs["loss"], "step": state.global_step})

            # validation 평가 및 loss 기록 (평가 주기에 따라 실행됨)
            if "eval_loss" in logs:
                wandb.log({"eval/loss": logs["eval_loss"], "step": state.global_step})


# TrainingArguments로 로그 빈도 및 기타 학습 설정 관리
training_args = TrainingArguments(
    output_dir="/tmp/clm-instruction-tuning",  # 출력 디렉터리 설정
    logging_steps=100,                         # 로그 빈도 설정 (매 100 스텝마다 로그 기록)
    evaluation_strategy="steps",               # 평가 전략을 'steps'로 설정
    eval_steps=100,                            # 평가 빈도 설정
    save_steps=0,                              # 저장 비활성화
    save_total_limit=0,                        # 체크포인트 개수 제한 없음
    save_strategy="no"                         # 'no'로 설정하여 저장 완전 비활성화
)

# Trainer 생성
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,                         # TrainingArguments로 설정 전달
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    callbacks=[WandbLoggingCallback()]          # 콜백 추가
)


trainer.train()


max_memory_allocated_gb = round(torch.cuda.max_memory_allocated(0) / 1024**3, 1)
print('Max Alloc:', max_memory_allocated_gb, 'GB')
wandb.log({"max_memory_allocated_gb": max_memory_allocated_gb})